## Calls to Dagster

In [ ]:
import time

from dagster_graphql import DagsterGraphQLClient, DagsterGraphQLClientError

from dagster import DagsterRunStatus


# Function to wait for completion
def wait_for_job_completion(run_id, timeout=3600, poll_interval=5):
    start_time = time.time()

    while True:
        # Check if timeout reached
        if time.time() - start_time > timeout:
            raise TimeoutError(f"Job {run_id} did not complete within {timeout} seconds")

        try:
            status = client.get_run_status(run_id)
            if status == DagsterRunStatus.SUCCESS:
                return True, status
            if status in [DagsterRunStatus.FAILURE, DagsterRunStatus.CANCELED]:
                return False, status
            print(f"Job status: {status}. Waiting...")
            time.sleep(poll_interval)

        except DagsterGraphQLClientError as exc:
            print(f"Error checking job status: {exc}")
            raise


def wait_until_model_ready(model_server_url, model_name, max_retries=10, timeout=5, initial_delay=5) -> None:
    """Wait until the model is ready."""

    # Initial delay to allow model server to start
    print(f"Waiting for {initial_delay} seconds for model server to start...")
    time.sleep(initial_delay)

    retry_count = 0
    while retry_count < max_retries:
        try:
            response = requests.get(f"{model_server_url}/v2/models/{model_name}/ready")
            response.raise_for_status()  # Raise an exception for bad status codes
            print("Ready")
            break
        except requests.exceptions.RequestException as e:
            if retry_count == max_retries - 1:
                print(f"Model is not ready after {max_retries} retries: {e}")
            else:
                print(f"Not ready, retrying in {timeout} seconds: {e}")
        time.sleep(timeout)
        retry_count += 1

In [ ]:
client = DagsterGraphQLClient("localhost", port_number=3000)

# Wait for the job to complete
try:
    new_run_id: str = client.submit_job_execution(
        "train_diabetes_model_job",  # Your job name
        repository_location_name="src.dagster_job.repository",  # Location from your error
        run_config={},
    )

    print(f"Job submitted with run_id: {new_run_id}")

    success, status = wait_for_job_completion(new_run_id)
    if success:
        print("Job completed successfully!")
    else:
        print(f"Job failed with status: {status}")

except Exception as e:
    print(f"Error while waiting for job completion: {e}")
    raise

In [ ]:
%%bash

export MODEL_VERSION=$(curl -s -X GET "http://localhost:5000/api/2.0/mlflow/registered-models/alias?name=diabetes-model&alias=dev" \
    | jq -r '.model_version.version')

echo "Model version: $MODEL_VERSION"

echo "Restarting docker with latest version"

docker-compose --profile model-server up -d  --force-recreate mlflow-diabetes-model


## Testing python calls

In [ ]:
import pandas as pd
import requests
from sklearn import datasets

model_server_url = "http://localhost:7001"
model_name = "mlflow-model"

model_server_url = "http://localhost:7000"
model_name = "diabetes-model"

In [ ]:
dataset = datasets.load_diabetes()

diabetes_result = pd.DataFrame(dataset["data"], columns=dataset["feature_names"])

Make prediction for all rows in a dataframe

In [ ]:
wait_until_model_ready(model_server_url, model_name)

In [ ]:
response = requests.post(
    f"{model_server_url}/invocations",
    json={
        "dataframe_split": {"columns": diabetes_result.columns.to_list(), "data": diabetes_result.to_numpy().tolist()},
    },
    timeout=5,
)
response_data = response.json()
# print(json.dumps(response_data, indent=4))

diabetes_result_with_predictions = diabetes_result.copy()
diabetes_result_with_predictions["predictions_response"] = response_data["predictions"]

diabetes_result_with_predictions

## Testing API calls to model server

#### Setup for Dagster built MLflow container

In [ ]:
# Config for diabetes-model-server
%env MODEL_SERVER_URL=http://localhost:7000
%env MODEL_NAME=diabetes-model

### Setup for Seldom mlserver

In [ ]:
# Config for mlserver (Seldon.IO container)
%env MODEL_SERVER_URL=http://localhost:8080
%env MODEL_NAME=diabetes-model

In [ ]:
%%bash

docker-compose up -d mlserver


### Calls to server

In [ ]:
%%bash

curl -f -s  "${MODEL_SERVER_URL}/v2/models/diabetes-model/ready" && echo "Ready" || echo "Not ready"


In [ ]:
%%bash
curl -s -X POST ${MODEL_SERVER_URL}/invocations \
 -H "Content-Type: application/json" \
  -d '{
        "dataframe_split": {
            "columns": ["age", "sex", "bmi", "bp", "s1", "s2", "s3", "s4", "s5", "s6"],
            "data": [
                [0.038076, 0.050680, 0.061696, 0.021872, -0.044223, -0.034821, -0.043401, -0.002592, 0.019907, -0.017646],
                [-0.001882, 0.352598, -0.170647, 0.190409, 0.273711, -0.024980, -0.091299, 0.042257, -0.049390, -0.092804]
            ]
        }
      }' \
  | jq



In [ ]:
%%bash

curl -X POST -s "${MODEL_SERVER_URL}/v2/models/${MODEL_NAME}/infer" \
     -H "Content-Type: application/json" \
     -d '{
           "inputs": [
             {
               "name": "input-0",
               "shape": [2, 10],
               "datatype": "FP64",
               "data": [
                    [0.038076, 0.050680, 0.061696, 0.021872, -0.044223, -0.034821, -0.043401, -0.002592, 0.019907, -0.017646],
                    [0.041708, 0.059182, 0.063738, 0.022681, -0.042640, -0.034450, -0.042857, -0.002639, 0.020058, -0.017646]
               ]
             }
           ]
         }' | jq


In [ ]:
%%bash
curl -X GET -s "${MODEL_SERVER_URL}/v2/models/${MODEL_NAME}" | jq


##### Force model unload/load with curl

In [ ]:
%%bash

curl -s  -X POST "${MODEL_SERVER_URL}/v2/repository/models/${MODEL_NAME}/unload"


In [ ]:
%%bash

curl -s -X POST "${MODEL_SERVER_URL}/v2/repository/models/${MODEL_NAME}/load" | jq


Check model status with repository index call

In [ ]:
%%bash

curl -s -X POST "${MODEL_SERVER_URL}/v2/repository/index" \
  -H "Content-Type: application/json" \
  -d "{}" | jq

#### Using Custom docker container model-server

In [ ]:
%%bash

curl -s -X POST "${MODEL_SERVER_URL}/invocations" \
 -H "Content-Type: application/json" \
  -d '{
        "dataframe_split": {
            "columns": ["age", "sex", "bmi", "bp", "s1", "s2", "s3", "s4", "s5", "s6"],
            "data": [
                [0.038076, 0.050680, 0.061696, 0.021872, -0.044223, -0.034821, -0.043401, -0.002592, 0.019907, -0.017646],
                [-0.001882, -0.044642, -0.118358, -0.034689, -0.045946, -0.034157, -0.072402, 0.042324, -0.009637, 0.034509]
            ]
        }
      }' \
  | jq


In [ ]:
%%bash

curl -s -X POST "${MODEL_SERVER_URL}/v2/repository/index" \
  -H "Content-Type: application/json" \
  -d "{}" | jq
